## Assignment 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
yeast = pd.read_csv('data/yeast.data', sep = '\s+', header = None)
col_names = ['v{}'.format(i) for i in range(0, 9)]
col_names.append('y') 
yeast.columns = col_names
yeast.rename(columns={yeast.columns[0]: 'Source'}, inplace = True)
yeast.head()

,Source,v1,v2,v3,v4,v5,v6,v7,v8,y
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [4]:
x = yeast.iloc[:, 1:9]
y = yeast['y']

## Ourlier Detection

Outlier detectot to be used:
- LOF: Local Outlier Factor
- One class SVM
- Isolation Forest

In [5]:
#sns.pairplot(yeast.iloc[:,1:10], hue = 'y')
#plt.savefig('pics/pair.png')

### One class SVM

In [6]:
from sklearn.svm import OneClassSVM
clf = OneClassSVM(gamma = 'auto')
outlier = clf.fit_predict(x)

In [7]:
pd.Series(outlier).value_counts()

 1    743
-1    741
dtype: int64

Any data labeled by -1 is treated as an outlier using the method. 

### LOF

In [8]:
from sklearn.neighbors import LocalOutlierFactor as LOF

In [9]:
lof = LOF()
outlier1 = lof.fit_predict(x)
pd.Series(outlier).value_counts()

/Users/yry/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


 1    743
-1    741
dtype: int64

### Isolation Forest

In [10]:
from sklearn.ensemble import IsolationForest

In [11]:
clf = IsolationForest()
outlier2 = clf.fit_predict(x)
pd.Series(outlier2).value_counts()

/Users/yry/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/yry/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/yry/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


 1    1335
-1     149
dtype: int64

In [12]:
np.where(outlier1 != outlier2)

(array([   5,   29,   41,   69,  118,  139,  148,  174,  207,  220,  227,
         236,  237,  239,  242,  244,  246,  272,  274,  318,  323,  325,
         326,  345,  376,  377,  421,  483,  491,  496,  508,  528,  533,
         538,  575,  579,  587,  595,  605,  614,  661,  664,  667,  692,
         702,  704,  722,  745,  746,  748,  755,  759,  778,  786,  787,
         836,  874,  875,  900,  923,  926,  954,  955,  962,  963,  971,
         977,  986,  987,  997,  998,  999, 1021, 1039, 1076, 1078, 1085,
        1105, 1110, 1114, 1120, 1143, 1149, 1163, 1188, 1192, 1195, 1213,
        1217, 1225, 1250, 1272, 1283, 1299, 1315, 1332, 1356, 1357, 1363,
        1364, 1382, 1386, 1396, 1429, 1434, 1446, 1448, 1455, 1465, 1466]),)

### Remove outlier

In [13]:
x_ = x.iloc[np.where(outlier2 == 1)[0],].reset_index(drop = True)
y_ = y.iloc[np.where(outlier2 == 1)[0],].reset_index(drop = True)

## Build up ANN model.

In [28]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

In [30]:
scalar = MinMaxScaler()
x_ = scalar.fit_transform(x_)
y_ = np.array(y_).reshape(-1,1)
enc = OneHotEncoder()
y_ = OneHotEncoder().fit_transform(y_)
y_ = y_.toarray()

In [31]:
x_tr, x_te, y_tr, y_te = train_test_split(x_, y_,  test_size = 0.34, random_state = 256)

In [39]:
MLP = Sequential([
    # Input layer:
    Dense(3, input_shape =(8, ), activation = tf.nn.sigmoid),
    Dense(3, activation = tf.nn.sigmoid),
    Dense(9, activation = tf.nn.sigmoid)
])

MLP.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
MLP.fit(x_tr,y_tr, epochs = 50)

Train on 881 samples
Epoch 1/50
881/881 [==============================] - 0s 384us/sample - loss: 0.2815 - accuracy: 0.3303
Epoch 2/50
881/881 [==============================] - 0s 37us/sample - loss: 0.2667 - accuracy: 0.3303
Epoch 3/50
881/881 [==============================] - 0s 35us/sample - loss: 0.2530 - accuracy: 0.3303
Epoch 4/50
881/881 [==============================] - 0s 36us/sample - loss: 0.2403 - accuracy: 0.3303
Epoch 5/50
881/881 [==============================] - 0s 38us/sample - loss: 0.2284 - accuracy: 0.3303
Epoch 6/50
881/881 [==============================] - 0s 40us/sample - loss: 0.2173 - accuracy: 0.3303
Epoch 7/50
881/881 [==============================] - 0s 37us/sample - loss: 0.2068 - accuracy: 0.3303
Epoch 8/50
881/881 [==============================] - 0s 32us/sample - loss: 0.1968 - accuracy: 0.3303
Epoch 9/50
881/881 [==============================] - 0s 30us/sample - loss: 0.1873 - accuracy: 0.3303
Epoch 10/50
881/881 [==============================